In [2]:
from collections import Counter
import math
import csv

#########################################################################################
##############################          Parametres         ##############################
#########################################################################################

#donnée de test Euterpe,7.0,96,0.215,0,.1731309869338593,2.346231774167983,1.9400263515307,1.583696215187637,.956641
#true pour afficher des information sur les données, false pour ne pas afficher
fonction_Info_Data=False
#True pour afficher les k planets les plus plosible au parametres, False pour ne pas afficher
fonction_KNN=True
#"Nombre de résultat"
k=1
#"Absolute magnitude parameter"
h = 7.0
#"object diameter (from equivalent sphere) km Unit"
diameter = 96
#"Geometric albedo"
albedo = 0.215
#"1-sigma uncertainty in object diameter km Unit"
diameter_sigma = 0
#"Eccentricity"
e = .1731309869338593
#"Semi-major axis au Unit"
a = 2.346231774167983
#"perihelion distance au Unit"
q = 1.9400263515307
#"inclination; angle with respect to x-y ecliptic plane"
i = 1.583696215187637
#"Minimum orbit intersection distance"
moid = .956641

#########################################################################################
#########################################################################################
#########################################################################################

#########################################################################################
######################    Fonction d'information sur les données   ######################
#########################################################################################

#Fonction qui retroune la moyenne, le min , le max d'une des colones du fichier csv donné en paramètre
#filename : le nom du fichier csv.
#column_number : le numéro de la colonne.
def calculate_statistics(filename, column_number):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        values = []
        for row in reader:
            if row[column_number] != '0':
                values.append(float(row[column_number]))
        if len(values) > 0:
            mean = sum(values) / len(values)
            minimum = min(values)
            maximum = max(values)
            return mean, minimum, maximum
        else:
            return None
        
#Fonction qui permet d'afficher les statistiques de chaque colonne du fichier csv
def print_statistics():
    with open('dataset.csv', newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)
        for i in range(len(header)):
            if i >= 5:
                stats = calculate_statistics('dataset.csv', i)
                if stats is not None:
                    mean, minimum, maximum = stats
                    print(f'{header[i]}: mean={mean:.2f}, min={minimum:.2f}, max={maximum:.2f}')

if(fonction_Info_Data==True):
    print_statistics()
     
#########################################################################################
#########################################################################################
#########################################################################################

#########################################################################################
#########################    Fonctions d'utilisation de KNN     #########################
#########################################################################################

#On mes les parametre dans un tableaux query pour pouvoir ensuite le reutiliser
query = [h, diameter, albedo, diameter_sigma, e, a, q, i, moid]

#Fonction qui permet de classer les données en fonction de leur similarité avec d'autres données connues
#data : un ensemble de données connues pour effectuer la classification.
#query : un point de données à classer.
#k : le nombre de voisins les plus proches à considérer lors de la classification.
#distance_fn : une fonction qui calcule la distance entre deux points de données.
#choice_fn : une fonction qui calcule la classe majoritaire des k plus proches voisins.
def knn(data, query, k, distance_fn, choice_fn):
    neighbor_distances_and_indices = []
    for index , example in enumerate(data):
        distance = distance_fn(example[:-1], query)
        neighbor_distances_and_indices.append((distance, index))
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k]
    k_nearest_labels = [data[i][-1] for distance, i in k_nearest_distances_and_indices]
    return k_nearest_distances_and_indices , choice_fn(k_nearest_labels) 

#Fonction qui permet de calculer la distance euclidienne entre deux points
#point1 : un point de données.
#point2 : un point de données.
def euclidean_distance(point1, point2):
    sum_squared_distance = 0
    for i in range(len(point1)):
        sum_squared_distance += math.pow(point1[i] - point2[i], 2)
    return math.sqrt(sum_squared_distance)

#Fonction qui traites les donner pour pouvoir les utiliser dans la fonction knn et qui affiche les k planets les plus plosible au parametres
#movie_query : les parametres de la planete a trouver
def mostSimilaPlanet(movie_query):
    raw_planet_data = []
    with open('dataset.csv', 'r') as md:
        next(md)
        for line in md.readlines():
            data_row = line.strip().split(',')
            raw_planet_data.append(data_row)
    planet_recommendation_data = []
    for row in raw_planet_data:
        data_row = list(map(float, row[5:]))
        planet_recommendation_data.append(data_row)
    recommendation_indices, _ = knn(
        planet_recommendation_data, movie_query, k,
        distance_fn=euclidean_distance, choice_fn=lambda x: None
    )
    planet_recommendations = []
    for _, index in recommendation_indices:
        planet_recommendations.append(raw_planet_data[index])
    for recommendation in planet_recommendations:
        print(recommendation[2])
if(fonction_KNN==True):
    mostSimilaPlanet(movie_query=query)

#########################################################################################
#########################################################################################
#########################################################################################

#########################################################################################
##########################   Fonctions utiles pour creation   ###########################
#########################################################################################
      
#Fonction qui permet de normaliser les données du fichier csv
def reform():
    with open('dataset.csv', 'r') as f:
        reader = csv.reader(f)
        your_list = list(reader)
        for i in range(len(your_list)):
            for j in range(len(your_list[i])):
                if(your_list[i][j] == ''):
                    your_list[i][j] = '0'
    with open('dataset.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(your_list)

    with open('dataset.csv', 'r') as f:
        lines = f.readlines()
    with open('dataset.csv', 'w') as f:
        for line in lines:
            if line.strip():
                f.write(line)

#########################################################################################
#########################################################################################
#########################################################################################

    27 Euterpe
